In [1]:
import pandas as pd
import numpy as np


# EDA

### Summary
- there are 248 patients
- patients have more than 1 record for each visit
  - there are ~1113 visit ids, so each patient visits around 4-5 times
- sampling random patient, there are around 200 proteins found in each of their sample visits

In [6]:
#load dataset
proteins = pd.read_csv("amp-parkinsons-disease-progression-prediction/train_proteins.csv")
peptides = pd.read_csv("amp-parkinsons-disease-progression-prediction/train_peptides.csv")
clinical = pd.read_csv("amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")

proteins.shape,peptides.shape,clinical.shape

((232741, 5), (981834, 6), (2615, 8))

In [7]:
proteins.sample(10)

,visit_id,visit_month,patient_id,UniProt,NPX
183515,20216_48,48,20216,P01780,7220.93
90644,40876_12,12,40876,Q02818,15242.30
187099,26104_48,48,26104,P02763,6456960.00
76654,16574_12,12,16574,P11142,82523.50
79193,20791_12,12,20791,P00736,75556.60
123883,27971_24,24,27971,P61916,203214.00
33257,56073_0,0,56073,P36980,10048.30
43549,7151_6,6,7151,O94919,31742.00
92640,44001_12,12,44001,P05156,98632.00
200416,58674_48,48,58674,P16035,65892.00


In [22]:
proteins.patient_id.nunique()

248

In [20]:
# number of unique visits
proteins.visit_id.nunique()
# average number of visits per patient
proteins.visit_id.nunique()/proteins.patient_id.nunique()

4.487903225806452

In [21]:
proteins.visit_month.nunique()

15

In [23]:
proteins[proteins['patient_id'] == 20216]['visit_month'].value_counts()

6     214
12    209
24    208
48    208
0     205
36    203
Name: visit_month, dtype: int64

# Data Preprocessing